In [1]:
def CNN4_JS(in_data):
    input2d = tf.reshape(in_data, [-1,max_len,30,6])
    conv11 = tf.layers.conv2d(
        inputs=input2d, 
        filters=32, 
        kernel_size=[3, 3], 
        strides=2,
        padding="same", 
        activation=tf.nn.relu)
    conv12 = tf.layers.conv2d(
        inputs=conv11, 
        filters=64, 
        kernel_size=[3, 3], 
        strides=2,
        padding="same", 
        activation=tf.nn.relu)
    conv21 = tf.layers.conv2d(
        inputs=conv12, 
        filters=128, 
        kernel_size=[3, 3], 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    conv22 = tf.layers.conv2d(
        inputs=conv21, 
        filters=256, 
        kernel_size=[3, 3], 
        strides=2,
        padding="same", 
        activation=tf.nn.relu)
    bn2 = tf.contrib.layers.batch_norm(conv22, center=True, scale=True, 
                                          is_training=True
                                          )
    conv31 = tf.layers.conv2d(
        inputs=bn2, 
        filters=256, 
        kernel_size=[3, 3], 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    bn31 = tf.contrib.layers.batch_norm(conv31, center=True, scale=True, 
                                          is_training=True
                                          )
    conv32 = tf.layers.conv2d(
        inputs=bn31, 
        filters=256, 
        kernel_size=[3, 3], 
        strides=2,
        padding="same", 
        activation=tf.nn.relu)
    bn32 = tf.contrib.layers.batch_norm(conv32, center=True, scale=True, 
                                          is_training=True)
    conv41 = tf.layers.conv2d(
        inputs=bn32, 
        filters=512, 
        kernel_size=[3, 3], 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    pool_flat = tf.reshape(conv41,[-1,32*2*512])#tf.layers.flatten(conv41)
    fc = tf.layers.dense(
        inputs= pool_flat, units=512, activation=tf.nn.relu)
    output = tf.layers.dense(inputs=fc, units=max_id+1)    
    return output
def CNNmodel(in_data):
    input2d = tf.reshape(in_data, [-1,max_len,30,6])
    conv1 = tf.layers.conv2d(
        inputs=input2d, 
        filters=20, 
        kernel_size=[5, 5], 
        strides=1,
        #padding="same", 
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, 
        pool_size=[2, 2], 
        strides=2)
        #padding="same")
    conv2 = tf.layers.conv2d(
        inputs=pool1, 
        filters=50, 
        kernel_size=[5, 5], 
        strides=1,
        #padding="same", 
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2, 
        pool_size=[2, 2], 
        strides=2)
        #padding="same")
    #pool_flat = tf.reshape(pool2, [-1, 4 * 4 * 50])
    pool_flat = tf.layers.flatten(pool2)
    fc = tf.layers.dense(
        inputs= pool_flat, units=500, activation=tf.nn.relu)
    output = tf.layers.dense(inputs=fc, units=max_id)    
    return output

def CNN_keras(in_data):
    input2d = tf.reshape(in_data, [-1,max_len,30,6])
    conv1 = tf.layers.conv2d(
        inputs=input2d, 
        filters=64, 
        kernel_size=[3, 3], 
        #padding="same", 
        activation=tf.nn.relu)    
    conv2 = tf.layers.conv2d(
        inputs=conv1, 
        filters=128, 
        kernel_size=[3, 3],
        #padding="same", 
        activation=tf.nn.relu)     
    pool1 = tf.layers.max_pooling2d(
        inputs=conv2, 
        pool_size=[2, 2],
        strides=1)
        #padding="same")    
    dropout1 = tf.layers.dropout(
        inputs=pool1,
        rate=0.3)
    pool_flat = tf.layers.flatten(dropout1)
    fc = tf.layers.dense(
        inputs= pool_flat, units=128, activation=tf.nn.relu)
    dropout2 = tf.layers.dropout(
        inputs=fc,
        rate=0.3)
    output = tf.layers.dense(inputs=dropout2, units=max_id)     
    return output

In [2]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip
from sklearn.model_selection import train_test_split

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
path_csi_hc = 'J:\\Data\\Wi-Fi_HC\\180_100\\'

# data info
df_info = pd.read_csv('data_subc_sig.csv')

# parameters
max_id = np.max(df_info['id_person'])
no_classes = max_id
#max_len = int(np.max(df_info['len']))
max_len = 500
learning_rate = 0.1
training_epochs = 10000
batch_size = 10


C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
file_list = os.listdir(path_csi_hc)
file = file_list[0]
data_read = np.load(path_csi_hc + file)

#filter outliers
xs_med = np.median(np.abs(data_read[:,4:]))
xs_std = np.std(np.abs(data_read[:,4:]))
xs_th = xs_med+xs_std

data_read_f =  np.array([row for row in list(data_read) if np.max(np.abs(row[4:])) <= xs_th])

# train test split
data_tr,data_te = train_test_split(data_read_f,test_size=0.2,random_state=10)
#input data
data_xs_tr = data_tr[:,4:].astype('float32') / xs_th
data_xs_te = data_te[:,4:].astype('float32') / xs_th
data_y_tr = data_tr[:,0].astype('int')
data_y_te = data_te[:,0].astype('int')

x_train = data_xs_tr.reshape([-1,500,30,6])
x_test = data_xs_te.reshape([-1,500,30,6])
y_train =  tf.keras.utils.to_categorical(data_y_tr, no_classes)
y_test =  tf.keras.utils.to_categorical(data_y_te, no_classes)

In [4]:
#a = data_read[data_read[:,0] <10]
#a.shape

In [5]:
# create the training datasets
dx_train = tf.data.Dataset.from_tensor_slices(x_train)
# apply a one-hot transformation to each label for use in the neural network
dy_train = tf.data.Dataset.from_tensor_slices(y_train)
# zip the x and y training data together and shuffle, batch etc.
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(50).repeat().batch(10)

# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,train_dataset.output_shapes)
next_element = iterator.get_next()

# make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)

In [6]:
# define cost/loss & optimizer
logits = CNN_keras(next_element[0])
labels_y = next_element[1]
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=labels_y))
    tf.summary.scalar('loss',loss)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    # Ensures that we execute the update_ops before performing the train_step
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# accuracy
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_pred'):
        pred = tf.argmax(logits,1)
        equal = tf.equal(pred,tf.argmax(labels_y,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(equal,tf.float32))
tf.summary.scalar('accuracy',accuracy)

#init
init_op = tf.global_variables_initializer()
training_init_op = iterator.make_initializer(train_dataset)
merged_summary_operation = tf.summary.merge_all()

In [7]:
# run the training
epochs = training_epochs
results = pd.DataFrame(columns=['loss','accuracy'])
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    train_summary_writer = tf.summary.FileWriter('./tmp/train',sess.graph)
    test_summary_writer = tf.summary.FileWriter('./tmp/test')
    for i in range(epochs):
        log,lab,pr = sess.run([logits,labels_y,pred])
        print(np.argmax(log,1))
        l, _, acc,merged_summary = sess.run([loss, optimizer, accuracy,merged_summary_operation])
        
        print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i+1, l, acc * 100))
        '''
        # now setup the validation run
        test_iters = 1
        # re-initialize the iterator, but this time with validation data
        sess.run(test_init_op)
        test_l, test_acc = sess.run([loss, accuracy])
        print("Epoch: {}, test loss: {:.3f}, test accuracy: {:.2f}%".format(i+1, test_l, test_acc * 100))
        '''
        results.loc[i] = [l,acc]
        train_summary_writer.add_summary(merged_summary)

[21 21 21 21 21 21 21 21 90 21]
Epoch: 1, loss: 4.699, training accuracy: 0.00%
[54 54 54 54 54 54 54 54 54 54]
Epoch: 2, loss: 12277789.000, training accuracy: 0.00%
[100 100 100 100 100 100 100 100 100 100]
Epoch: 3, loss: 507331.250, training accuracy: 0.00%
[77 77 77 77 77 77 77 77 77 77]
Epoch: 4, loss: 5203841.500, training accuracy: 0.00%
[2 2 2 2 2 2 2 2 2 2]
Epoch: 5, loss: 321672.562, training accuracy: 0.00%
[39 39 39 39 39 39 39 39 39 39]
Epoch: 6, loss: 317702.312, training accuracy: 0.00%
[74 74 74 74 74 74 74 74 74 74]
Epoch: 7, loss: 108006.172, training accuracy: 0.00%
[51 51 51 51 51 46 51 51 51 51]
Epoch: 8, loss: 5001.162, training accuracy: 0.00%
[33 33 33 33 33 33 33 33 33 33]
Epoch: 9, loss: 2368.286, training accuracy: 0.00%
[75 75 75 75 75 75 75 75 75 75]
Epoch: 10, loss: 520490.344, training accuracy: 0.00%
[81 81 81 81 81 81 81 81 81 81]
Epoch: 11, loss: 5760.351, training accuracy: 0.00%
[29 29 29 29 29 29 29 29 29 29]
Epoch: 12, loss: 3553.363, training acc

Epoch: 102, loss: 4.734, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 103, loss: 4.771, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 104, loss: 4.509, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 105, loss: 4.940, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 106, loss: 4.760, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 107, loss: 4.669, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 108, loss: 4.743, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 109, loss: 4.688, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 110, loss: 4.740, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 111, loss: 4.640, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 112, loss: 4.911, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 113, loss: 4.651, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 114, loss

Epoch: 202, loss: 4.573, training accuracy: 0.00%
[62 62 62 62 62 62 62 62 62 62]
Epoch: 203, loss: 4.708, training accuracy: 0.00%
[62 62 62 62 62 62 62 62 62 62]
Epoch: 204, loss: 4.767, training accuracy: 0.00%
[64 64 64 64 64 64 64 64 64 64]
Epoch: 205, loss: 4.695, training accuracy: 0.00%
[64 64 64 64 64 64 64 64 64 64]
Epoch: 206, loss: 4.751, training accuracy: 0.00%
[64 64 64 64 64 64 64 64 64 64]
Epoch: 207, loss: 4.602, training accuracy: 0.00%
[64 64 64 64 64 64 64 64 64 64]
Epoch: 208, loss: 4.770, training accuracy: 0.00%
[64 64 64 64 64 64 64 64 64 64]
Epoch: 209, loss: 4.555, training accuracy: 0.00%
[64 64 64 64 64 64 64 64 64 64]
Epoch: 210, loss: 4.473, training accuracy: 0.00%
[73 73 73 73 73 73 73 73 73 73]
Epoch: 211, loss: 4.648, training accuracy: 0.00%
[73 73 73 73 73 73 73 73 73 73]
Epoch: 212, loss: 4.785, training accuracy: 0.00%
[73 73 73 73 73 73 73 73 73 73]
Epoch: 213, loss: 4.626, training accuracy: 0.00%
[73 73 73 73 73 73 73 73 73 73]
Epoch: 214, loss

Epoch: 302, loss: 4.683, training accuracy: 0.00%
[86 86 86 86 86 86 86 86 86 86]
Epoch: 303, loss: 4.738, training accuracy: 0.00%
[86 86 86 86 86 86 86 86 86 86]
Epoch: 304, loss: 4.489, training accuracy: 0.00%
[86 86 86 86 86 86 86 86 86 86]
Epoch: 305, loss: 4.603, training accuracy: 10.00%
[86 86 86 86 86 86 86 86 86 86]
Epoch: 306, loss: 4.791, training accuracy: 0.00%
[76 76 76 76 76 76 76 76 76 76]
Epoch: 307, loss: 4.619, training accuracy: 0.00%
[76 76 76 76 76 76 76 76 76 76]
Epoch: 308, loss: 4.578, training accuracy: 0.00%
[76 76 76 76 76 76 76 76 76 76]
Epoch: 309, loss: 4.683, training accuracy: 0.00%
[76 76 76 76 76 76 76 76 76 76]
Epoch: 310, loss: 4.666, training accuracy: 0.00%
[76 76 76 76 76 76 76 76 76 76]
Epoch: 311, loss: 4.636, training accuracy: 0.00%
[76 76 76 76 76 76 76 76 76 76]
Epoch: 312, loss: 4.808, training accuracy: 0.00%
[76 76 76 76 76 76 76 76 76 76]
Epoch: 313, loss: 4.773, training accuracy: 0.00%
[76 76 76 76 76 76 76 76 76 76]
Epoch: 314, los

Epoch: 402, loss: 4.672, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 403, loss: 4.555, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 404, loss: 4.762, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 405, loss: 4.560, training accuracy: 10.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 406, loss: 4.648, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 407, loss: 4.668, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 408, loss: 4.757, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 409, loss: 4.725, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 410, loss: 4.641, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 411, loss: 4.581, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 412, loss: 4.525, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 413, loss: 4.683, training accuracy: 0.00%
[14 14 14 14 14 14 14 14 14 14]
Epoch: 414, los

...

Epoch: 1606, loss: 4.686, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1607, loss: 4.761, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1608, loss: 4.644, training accuracy: 10.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1609, loss: 4.729, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1610, loss: 4.604, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1611, loss: 4.555, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1612, loss: 4.606, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1613, loss: 4.502, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1614, loss: 4.879, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1615, loss: 4.673, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1616, loss: 4.563, training accuracy: 0.00%
[55 55 55 55 55 55 55 55 55 55]
Epoch: 1617, loss: 4.684, training accuracy: 0.00%
[86 86 86 86 86 86 86 86 86 86]
Epo

Epoch: 1706, loss: 4.829, training accuracy: 0.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1707, loss: 4.684, training accuracy: 0.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1708, loss: 4.660, training accuracy: 10.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1709, loss: 4.765, training accuracy: 0.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1710, loss: 4.659, training accuracy: 0.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1711, loss: 4.675, training accuracy: 0.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1712, loss: 4.747, training accuracy: 10.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1713, loss: 4.639, training accuracy: 0.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1714, loss: 4.529, training accuracy: 0.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1715, loss: 4.558, training accuracy: 0.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1716, loss: 4.719, training accuracy: 0.00%
[78 78 78 78 78 78 78 78 78 78]
Epoch: 1717, loss: 4.877, training accuracy: 0.00%
[18 18 18 18 18 18 18 18 18 18]
Ep

Epoch: 1806, loss: 4.794, training accuracy: 0.00%
[35 35 35 35 35 35 35 35 35 35]
Epoch: 1807, loss: 4.894, training accuracy: 0.00%
[35 35 35 35 35 35 35 35 35 35]
Epoch: 1808, loss: 4.734, training accuracy: 10.00%
[35 35 35 35 35 35 35 35 35 35]
Epoch: 1809, loss: 4.670, training accuracy: 0.00%
[35 35 35 35 35 35 35 35 35 35]
Epoch: 1810, loss: 4.682, training accuracy: 0.00%
[98 98 98 98 98 98 98 98 98 98]
Epoch: 1811, loss: 4.588, training accuracy: 0.00%
[98 98 98 98 98 98 98 98 98 98]
Epoch: 1812, loss: 4.673, training accuracy: 0.00%
[98 98 98 98 98 98 98 98 98 98]
Epoch: 1813, loss: 4.568, training accuracy: 0.00%
[66 66 66 66 66 66 66 66 66 66]
Epoch: 1814, loss: 4.529, training accuracy: 0.00%
[66 66 66 66 66 66 66 66 66 66]
Epoch: 1815, loss: 4.539, training accuracy: 0.00%
[66 66 66 66 66 66 66 66 66 66]
Epoch: 1816, loss: 4.754, training accuracy: 0.00%
[66 66 66 66 66 66 66 66 66 66]
Epoch: 1817, loss: 4.607, training accuracy: 0.00%
[66 66 66 66 66 66 66 66 66 66]
Epo

Epoch: 1904, loss: 4.806, training accuracy: 0.00%
[11 11 11 11 11 11 11 11 11 11]
Epoch: 1905, loss: 4.571, training accuracy: 0.00%
[79 79 79 79 79 79 79 79 79 79]
Epoch: 1906, loss: 4.743, training accuracy: 0.00%
[79 79 79 79 79 79 79 79 79 79]
Epoch: 1907, loss: 4.834, training accuracy: 0.00%
[79 79 79 79 79 79 79 79 79 79]
Epoch: 1908, loss: 4.701, training accuracy: 0.00%
[79 79 79 79 79 79 79 79 79 79]
Epoch: 1909, loss: 4.697, training accuracy: 0.00%
[79 79 79 79 79 79 79 79 79 79]
Epoch: 1910, loss: 4.750, training accuracy: 0.00%
[79 79 79 79 79 79 79 79 79 79]
Epoch: 1911, loss: 4.682, training accuracy: 0.00%
[79 79 79 79 79 79 79 79 79 79]
Epoch: 1912, loss: 4.627, training accuracy: 0.00%
[79 79 79 79 79 79 79 79 79 79]
Epoch: 1913, loss: 4.716, training accuracy: 0.00%
[79 79 79 79 79 79 79 79 79 79]
Epoch: 1914, loss: 4.606, training accuracy: 0.00%
[48 48 48 48 48 48 48 48 48 48]
Epoch: 1915, loss: 4.714, training accuracy: 0.00%
[48 48 48 48 48 48 48 48 48 48]
Epoc

KeyboardInterrupt: 

In [ ]:
log.shape

In [ ]:
lab.shape

In [ ]:
lab[0]

In [ ]:
np.argmax(log,1)

In [ ]:
np.argmax(lab,1)